<a href="https://colab.research.google.com/github/hariseldon99/msph402b/blob/main/Root_Finding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Example Codes for Computational Linear Algebra
===================================================

**Examples for Gaussian Method**
* From file [gauss_method_ex.py](03-Computational_Linear_Algebra/gauss_method_ex.py)

#### The Problem:

Use the Gaussian elimination method to solve the following sets of linear equations.

\begin{align*}
25x + 5y + z &= 106.8 \\
64x + 8y + z &= 177.2 \\
144x + 12y + z &=279.2,
\end{align*}

and

\begin{align*}
12x + 10y - 7z &= 15 \\
6x + 5y + 3z &=  4\\
5x - y + 5z &= 9
\end{align*}

In [2]:
import numpy as np

def GEPP(A, b, doPP = True):
    '''
    Gaussian elimination with partial pivoting.
    
    input: A is an n x n numpy matrix
           b is an n x 1 numpy array
    output: x is the solution of Ax=b 
            with the entries permuted in 
            accordance with the pivoting 
            done by the algorithm
    post-condition: A and b have been modified.
    '''
    n = len(A)
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between"+
                         "A & b.", b.size, n)
    # k represents the current pivot row. Since GE traverses the matrix in the 
    # upper right triangle, we also use k for indicating the k-th diagonal 
    # column index.
    
    # Elimination
    for k in range(n-1):
        if doPP:
            # Pivot
            maxindex = abs(A[k:,k]).argmax() + k
            if A[maxindex, k] == 0:
                raise ValueError("Matrix is singular.")
            # Swap
            if maxindex != k:
                A[[k,maxindex]] = A[[maxindex, k]]
                b[[k,maxindex]] = b[[maxindex, k]]
        else:
            if A[k, k] == 0:
                raise ValueError("Pivot element is zero. Try setting doPP to True.")
        #Eliminate
        for row in range(k+1, n):
            multiplier = A[row,k]/A[k,k]
            A[row, k:] = A[row, k:] - multiplier * A[k, k:]
            b[row] = b[row] - multiplier*b[k]
    # Back Substitution
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:],x[k+1:]))/A[k,k]
    return x

def det(A):
  _= GEPP(A, np.ones(A.shape[0]), doPP=True)
  return np.prod(np.diagonal(A))


print("Defined")

Defined


In [56]:
A = np.array([[25., 5., 1.],
              [64., 8., 1.],
              [144., 12., 1.]])

b = np.array([106.8,
              177.2,
              279.2])

x = GEPP(np.copy(A),np.copy(b), doPP=False)
print("First solution is given by x =", x)
print("Error is ", np.linalg.norm(A@x - b) * 100/np.linalg.norm(b), "%")

print("Determinant of first matrix is ", det(np.copy(A)))

A = np.array([[12., 10., -7.],
              [6., 5., 3.],
              [5., -1., 5.]])

b = np.array([15.,
              4.,
              9.])

try:
  x = GEPP(np.copy(A),np.copy(b), doPP=False)
except ValueError:
  x = GEPP(np.copy(A),np.copy(b))
print("Second solution is given by x =", x)
print("Error is ", np.linalg.norm(A@x - b) * 100/np.linalg.norm(b), "%")

First solution is given by x = [ 0.29047619 19.69047619  1.08571429]
Error is  3.372216160495845e-14 %
Determinant of first matrix is  -83.99999999999999
Second solution is given by x = [ 2.06699752 -1.3573201  -0.53846154]
Error is  2.334733358973876e-14 %


**Examples for LU Decomposition Method**
* From file [lu_decomp_ex.py](03-Computational_Linear_Algebra/lu_decomp_ex.py)

#### The Problem:

Solve the first of two systems of linear equations in the previous problem using the LU decomposition method.

In [77]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve
A = np.array([[25., 5., 1.],
              [64., 8., 1.],
              [144., 12., 1.]])

b = np.array([106.8,
              177.2,
              279.2])

A_fact, piv = lu_factor(A.copy())

print("Decomposed L Matrix:\n", np.tril(A_fact, k=0))
print("\n\nDecomposed U Matrix:\n", np.triu(A_fact, k=1))

x = lu_solve((A_fact.copy(), piv),b)

print("\nSolution is x =", x)
print("Solution is close?", np.allclose(A @ x, b))

id = np.eye(A.shape[0])
A_inv = np.zeros_like(A)

for i,row in enumerate(id):
  A_inv[:,i] = lu_solve((A_fact.copy(), piv),row)

print("\n\nInverse of matrix is:\n", A_inv)
print("Solution is close?", np.allclose(A @ A_inv, id))



Decomposed L Matrix:
 [[144.           0.           0.        ]
 [  0.17361111   2.91666667   0.        ]
 [  0.44444444   0.91428571  -0.2       ]]


Decomposed U Matrix:
 [[ 0.         12.          1.        ]
 [ 0.          0.          0.82638889]
 [ 0.          0.          0.        ]]

Solution is x = [ 0.29047619 19.69047619  1.08571429]
Solution is close? True


Inverse of matrix is:
 [[ 0.04761905 -0.08333333  0.03571429]
 [-0.95238095  1.41666667 -0.46428571]
 [ 4.57142857 -5.          1.42857143]]
Solution is close? True


**Examples for Gauss-Seidel Method**
* From file [gauss_seidel_ex.py](03-Computational_Linear_Algebra/gauss_method_ex.py)

#### The Problem:

Solve the following system of linear equations using the Gauss-Seidel method, displaying the relative error at each iteration until the solution approaches a fixed point within a small tolerance.

\begin{align*}
16x + 3y &= 11\\
7x - 11y &= 13.
\end{align*}

Repeat this method for the system of linear equations from the previous problem and show that the method fails.

In [3]:
import numpy as np

def gauss_seidel(A, b, tolerance=1e-10, max_iterations=100000, verbose=False):
    """
    Simple Function for the Gauss-Seidel Method for solving a system of linear equations
    Returns a numpy array consisting of the solution x, where A . x = b  

            Parameters:
                    A (numpy array): A square matrix of coefficients
                    b (numpy array): The RHS vector of the linear system

            Returns:
                    x (numpy array): Solution to the equation A . x = b 
    """
    x = np.zeros_like(b, dtype=np.double)
    
    if verbose:
      print("Iteration\t% soln: Relative err")
    #Iterate
    for k in range(max_iterations):
        
        x_old  = x.copy()
        
        #Loop over rows
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
        
        error = np.linalg.norm(x - x_old, ord=np.inf) / np.linalg.norm(x, ord=np.inf)
        if verbose:
          print("%lf\t %1.4lf " % (k, error * 100))
        
        #Stop condition 
        if error < tolerance:
            break
    if k == max_iterations -1:
      raise StopIteration("Error: Maximum iterations exceeded", k)
    
    return x

def isDDM(m, n):
  """
  Checks a numpy 2d square array for diagonal dominance
  """
 
  # for each row
  for i in range(0, n):   
    # for each column, finding sum of each row sans the diagonal
    sum = np.sum(m[i]) - np.abs(m[i,i])  
    if (abs(m[i,i]) < sum) :
        return False
 
  return True

print("Defined!")

Defined!


In [33]:
mat = np.array([[16, 3],
                [7, -11]])

rhs = np.array([11,
                13])

print("x =", gauss_seidel(mat, rhs, verbose=True))

Iteration	% soln: Relative err
0.000000	 100.0000 
1.000000	 16.8742 
2.000000	 2.0548 
3.000000	 0.2446 
4.000000	 0.0292 
5.000000	 0.0035 
6.000000	 0.0004 
7.000000	 0.0000 
8.000000	 0.0000 
9.000000	 0.0000 
10.000000	 0.0000 
11.000000	 0.0000 
12.000000	 0.0000 
x = [ 0.81218274 -0.66497462]


In [34]:
mat = np.array([[25, 5, 1],
                [64, 8, 1],
                [144, 12, 1]])

rhs = np.array([106.8,
                177.2,
                279.2])

print("Is matrix diagonally dominant?", isDDM(mat, 3))
print("x =", gauss_seidel(mat, rhs))

Is matrix diagonally dominant? False


StopIteration: ignored

#### The Problem:

Apply the previous algorithm for the Gauss-Seidel method in order to solve the following system of linear equations. 
\begin{align*}
12x + 3y - 5z &=1\\
x + 5y + 3z &=28\\
3x + 7y + 13z &=76
\end{align*}

Check for diagonal dominance before you run the solution algorithm. Repeat the same with the first two equations interchanged and observe how the loss of diagonal dominance leads to the failure of the Gauss-Seidel method, despite the fact that the system of equations have not fundamentally changed.


In [4]:
mat = np.array([[12, 3, -5],
                [1,  5,  3],
                [3,  7, 13]])

rhs = np.array([1,
                28,
                76])

print("Is matrix diagonally dominant?", isDDM(mat, 3))
print("x =", gauss_seidel(mat, rhs))

Is matrix diagonally dominant? True
x = [1. 3. 4.]


In [5]:
mat = np.array([[1,  5,  3],
                [12, 3, -5],
                [3,  7, 13]])

rhs = np.array([28,
                1,
                76])

print("Is matrix diagonally dominant?", isDDM(mat, 3))

try:
  print("x =", gauss_seidel(mat, rhs))
except Exception:
  print("The algorithm failed to converge")


Is matrix diagonally dominant? False


/tmp/ipykernel_42343/2503164286.py:28: RuntimeWarning: invalid value encountered in double_scalars
  error = np.linalg.norm(x - x_old, ord=np.inf) / np.linalg.norm(x, ord=np.inf)


The algorithm failed to converge
